In [1]:
import pandas as pd
from cassandra.cluster import Cluster

cluster = Cluster(['192.168.7.110', '192.168.7.33'])
#session = cluster.connect('data.storageinfo')
session = cluster.connect()

rows = session.execute('select loc from data.storageinfo limit 5')
for row in rows:
    print(row.loc.split('\u0001'))
    #print(type(row.loc))
    #print(row.loc)

cluster.shutdown()

['192.168.8.74', '80', '/PIIFormTest/form1.tar', 'CITY']
['192.168.8.74', '80', '/PIIFormTest/form1.tar', 'CITY']
['192.168.8.74', '80', '/PIIFormTest/form1.tar', 'CITY']
['192.168.8.74', '80', '/PIIFormTest/form4.pdf', 'SOCIAL_SECURITY_NUMBER']
['192.168.8.74', '80', '/PIIFormTest/form1.pdf', 'STREET_NUMBER_AND_NAME']


In [12]:
from cassandra.cluster import Cluster

def cs_insert(cs_host_list, tab_n, val_dict):
    cluster = Cluster(cs_host_list)
    session = cluster.connect()
    key_str = "(" + ",".join([key for key in val_dict.keys()]) + ")"
    val_str = "(" + ",".join(["%s" for val in val_dict.values()]) + ")"
    cql_str = "insert into " + tab_n + " " + key_str + " values " + val_str
    cql_val_tuple = tuple(val_dict.values())
    res = session.execute(cql_str, cql_val_tuple)
    session.shutdown()
    cluster.shutdown()

new_fs_dict = {"storage_type": 2,
                "ip": "192.168.7.11", "port": 3306,
                "dbs": 0, "tables": 0, "files": 0
                }

#cs_insert(['192.168.7.21'], "api.ds_services", new_fs_dict)

In [4]:
from cassandra.cluster import Cluster

def cs_select(cs_host_list, csql_str):
    cluster = Cluster(cs_host_list)
    session = cluster.connect()
    res = session.execute(csql_str)
    cluster.shutdown()
    return res
    
#rows = cs_select(['192.168.7.110', '192.168.7.111'], "select * from api.ds_services where ip='192.168.8.74' and port=1433")
#[print(row.ip, type(row.files), type(row.badges), row.connection, row.port) for row in rows]

DS_DELIMITER = "\u0001"
sel_ip = "192.168.7.71"
sel_port = 1433
sel_l = DS_DELIMITER.join([sel_ip, str(sel_port)]) + DS_DELIMITER
print(sel_l)
sel_u = DS_DELIMITER.join([sel_ip, str(sel_port+1)]) + DS_DELIMITER
print(sel_u)

#query = "select * from scanner.sqlcatalog"# where loc LIKE '%192.168.7.71%'"
#rows = cs_select(['192.168.7.21', '192.168.7.22'], query)
#[':'.join([row.db_name, row.table_name, row.col_name]) for row in rows if sel in row.loc]

#query = "select * from scanner.sqlcatalog where loc >= '" + sel_l + "' and loc < '" + sel_u + "' allow filtering"
#rows = cs_select(['192.168.7.110', '192.168.7.111'], query)
#[':'.join([row.ip, str(row.port), row.db_name, row.table_name, row.col_name]) for row in rows]

rows = cs_select(["192.168.9.91"], 'select count(entity_id) as piis from api.data_map group by entity_id, ddn_id')
#piis = len(set(row.entity_id for row in rows))
piis = sum([row.piis for row in rows])

print(piis)

#rows = cs_select(['192.168.7.110', '192.168.7.111'], 'select sum(dbs) as dbs, sum(tables) as tables, sum(files) as files from api.ds_services')
#[row.dbs for row in rows][0]

#rows = cs_select(['192.168.7.110', '192.168.7.111'], 'select count(ip) as servers, sum(services) as services from api.ds_servers')
#[row.servers for row in rows][0]
#servers, services = [(row.servers, row.services) for row in rows][0]
#print(servers, services)
#[print(type(row.ip)) for row in rows]

192.168.7.711433
192.168.7.711434
322


In [16]:
#query = "select entity_id from data.entity group by entity_id"
#query = "select count(entity_id) as count from data.entity group by entity_id limit 1"
query = "select count(entity_id) as count from data.entity limit 1"


rows = cs_select(['192.168.7.110', '192.168.7.111'], query)
entity_id_list = [row.count for row in rows]
print(entity_id_list)
#piis = len(entity_id_list)
#print(piis)

OperationTimedOut: errors={<Host: 192.168.7.33 datacenter1>: ConnectionException('Host has been marked down or removed'), <Host: 192.168.7.62 datacenter1>: ConnectionException('Host has been marked down or removed'), <Host: 192.168.7.22 datacenter1>: ConnectionException('Host has been marked down or removed'), <Host: 192.168.7.61 datacenter1>: ConnectionException('Host has been marked down or removed')}, last_host=192.168.7.110

In [18]:
#import sys
#query = "select ddn_id from api.ddn_binary group by ddn_id"
#rows = cs_select(['192.168.7.110', '192.168.7.111'], query)
#ddn_id_list = [row.ddn_id for row in rows]
#print(ddn_id_list)

ddn_id = '5b382afdbbb1c127acf105a7d45041c4'
query = "select * from api.ddn_binary"
#query = "select * from api.ddn_binary where ddn_id ='" + ddn_id + "' order by log_timestamp desc limit 1"
print(query)
rows = cs_select(['192.168.7.110', '192.168.7.111'], query)
ddn_pb_list = [row.ddn_binary for row in rows]
print(len(ddn_pb_list))
print(sys.getsizeof(ddn_pb_list[4]))
#print(ddn_pb_list[0])

select * from api.ddn_binary
6
13489312


In [15]:
#query = "select * from api.story_board"
query = "select * from scanner.hashinfo"


rows = cs_select(['192.168.7.110', '192.168.7.111'], query)
cs_data = [{"tf_loc": row.tf_loc} for row in rows]

#print(cs_data)
print(len(cs_data))

OperationTimedOut: errors={<Host: 192.168.7.21 datacenter1>: ConnectionException('Host has been marked down or removed'), <Host: 192.168.7.33 datacenter1>: ConnectionException('Host has been marked down or removed'), <Host: 192.168.7.22 datacenter1>: ConnectionException('Host has been marked down or removed'), <Host: 192.168.7.111 datacenter1>: ConnectionException('Host has been marked down or removed')}, last_host=192.168.7.110

In [31]:
#query = "select * from api.ds_services where ip='192.168.7.155' and port=1433 allow filtering"
query = "select * from api.ds_services where storage_type in (1,2)"

rows = cs_select(['192.168.7.110', '192.168.7.111'], query)
cs_data = [{"ip": row.ip,
            "port": row.port,
            "storage_type": row.storage_type} for row in rows]
print(cs_data)

[{'ip': '192.168.8.71', 'port': 1433, 'storage_type': 1}, {'ip': '192.168.7.155', 'port': 1433, 'storage_type': 1}, {'ip': '192.168.8.74', 'port': 3306, 'storage_type': 2}, {'ip': '192.168.8.71', 'port': 3306, 'storage_type': 2}, {'ip': '192.168.7.74', 'port': 3306, 'storage_type': 2}]


In [3]:
query = "select md5, source_uuid, loc from scanner.hashinfo limit 5"
rows = cs_select(['192.168.7.110', '192.168.7.111'], query)
cs_data = [{"md5": row.md5,
            "source_uuid": row.source_uuid,
            "loc": row.loc} for row in rows]
print(cs_data)

[{'md5': '82161242827b703e6acf9c726942a1e4', 'source_uuid': '9eae4195-b2ae-4d76-83c5-38d29aa1ec83', 'loc': '{"LastScanTime":"0001-01-01T00:00:00Z","Table":"userinfo9","Column":"uid","NormalizedColumn":"uid"}'}, {'md5': '0a09c8844ba8f0936c20bd791130d6b6', 'source_uuid': '9eae4195-b2ae-4d76-83c5-38d29aa1ec83', 'loc': '{"LastScanTime":"0001-01-01T00:00:00Z","Table":"userinfo9","Column":"uid","NormalizedColumn":"uid"}'}, {'md5': 'c4b31ce7d95c75ca70d50c19aef08bf1', 'source_uuid': '9eae4195-b2ae-4d76-83c5-38d29aa1ec83', 'loc': '{"LastScanTime":"0001-01-01T00:00:00Z","Table":"userinfo9","Column":"uid","NormalizedColumn":"uid"}'}, {'md5': '08ced3df2d196884401c4535cae59722', 'source_uuid': '7bc4ef46-93ab-489a-a575-d84da4a72cc9', 'loc': '{"LastScanTime":"0001-01-01T00:00:00Z","Table":"employee","Column":"mail","NormalizedColumn":"email"}'}, {'md5': 'bfc628eb1dc363d228e8d1e9e1e56bfa', 'source_uuid': '7bc4ef46-93ab-489a-a575-d84da4a72cc9', 'loc': '{"LastScanTime":"0001-01-01T00:00:00Z","Table":"em

In [15]:
#query = "select ddn_name, city, count(pii_record) as value from api.content_locations group by ddn_name, city"
query = "select ddn_id, city, count(last_seen_time) as value from api.access_locations group by ddn_id, city"
#query = "select city, pii_record from api.content_locations where ddn_name = 'aaa' allow filtering"
rows = cs_select(['192.168.7.110', '192.168.7.111'], query)
cs_data = [{"ddn_name": row.ddn_id,
            "content": row.city,
            "value": row.value} for row in rows]
print(cs_data)
#print(cs_data)
#if cs_data:
#    print("aaa")
#else:
#    print("bbb")
cs_data = [
    {"ddn_name": "hr", "content": "Beijing", "value": 5},
    {"ddn_name": "hr", "content": "Tianjin", "value": 1},
    {"ddn_name": "sales", "content": "Shanghai", "value": 5},
    {"ddn_name": "sales", "content": "Beijing", "value": 22},
    {"ddn_name": "hr", "content": "Nanjing", "value": 1},
]

df = pd.DataFrame(cs_data)
d = df.groupby(['content'])['value'].sum().reset_index().sort_values(by='value', ascending=False).set_index('content').T.to_dict()
#d = d.sort_values(by='value', ascending=False)
print(d)
print(type(d))

#.sort_values(by='value', ascending=False).to_dict()
cs_data = [{"content": k, "value": v['value']} for k, v in d.items()]
cs_data

[]
[{'ddn_name': '290c332d1efec714faa4bb975a075ae8', 'content': 'San Francisco,US', 'value': 3}, {'ddn_name': '54d752f95f5627ffbf66731317db0d20', 'content': 'internal,internal', 'value': 6}, {'ddn_name': '290c332d1efec714faa4bb975a075ae8', 'content': 'internal,internal', 'value': 5}, {'ddn_name': '54d752f95f5627ffbf66731317db0d20', 'content': 'San Francisco,US', 'value': 2}]
{'Beijing': {'value': 27}, 'Shanghai': {'value': 5}, 'Nanjing': {'value': 1}, 'Tianjin': {'value': 1}}
<class 'dict'>


[{'content': 'Beijing', 'value': 27},
 {'content': 'Shanghai', 'value': 5},
 {'content': 'Nanjing', 'value': 1},
 {'content': 'Tianjin', 'value': 1}]

In [5]:
query = "select ddn_name, feature_name as content, feature_value as value from api.pii_special"
cs_data = [{"content": row.content,
            "value": row.value} for row in rows]
cs_data

[]

In [43]:
#query = "select ddn_id as content, bytes as x, frequency as y from api.ddn_vol_freq"
#query = "select ddn_id, ddn_name as content, bytes as x, frequency as y from api.ddn_vol_freq limit 5000"
query = "select ddn_id, tf_loc, storage_type as content, sum(bytes) as x, sum(frequency) as y from api.ddn_vol_freq group by ddn_id, tf_loc, storage_type"

rows = cs_select(['192.168.7.110', '192.168.7.111'], query)
cs_data = [{"content": row.content,
            "x": row.x,
            "y": row.y} for row in rows]
#cs_data = [{"content": row.content,
#            "value": row.value} for row in rows]
print(cs_data)
#cs_data = [
#    {"content": "hr", "x": 5, "y": 5},
#    {"content": "hr", "x": 1, "y": 5},
#    {"content": "sales", "x": 5, "y": 5},
#    {"content": "sales", "x": 2, "y": 5},
#    {"content": "hr", "x": 1, "y": 5},
#]
#df = pd.DataFrame(cs_data)
#dx = df.groupby(['content'])['x'].sum().to_dict()
#dx
#dy = df.groupby(['content'])['y'].sum().to_dict()
#dy
#d = df.groupby(['content'])['x', 'y'].sum().to_dict()
#d
#{'x': {'hr': 7, 'sales': 7}, 'y': {'hr': 15, 'sales': 10}}
#keys = list(d['x'].keys())
#keys
#cs_data = [{"content": k, "value": v} for k, v in d.items()]

#cs_data = [{"content": key, "x": d['x'][key], "y": d['y'][key]} for key in keys]
#cs_data

[{'content': 2, 'x': 697436667904.0, 'y': 8973680}, {'content': 2, 'x': 1097005793280.0, 'y': 14114748}]


In [9]:
#query = "select ip, port from api.ds_services where ip='192.168.7.71' and port=80"
query = "select ip, port from api.ds_services where port=80 allow filtering"
rows = cs_select(['192.168.7.110', '192.168.7.111'], query)
[print(row.ip, row.port) for row in rows]

192.168.8.71 80
192.168.7.71 80


[None, None]

In [10]:
def cs_update(cs_host_list, tab_n, upg_d, sel_d):
    cluster = Cluster(cs_host_list)
    session = cluster.connect()
    upg_str = ",".join(
        [k + "='" + v + "'" if isinstance(v, str) \
         else k + "=" + str(v) if isinstance(v, int) \
         else k + "=" + str(v) \
         for k, v in upg_d.items()])
    sel_str = " and ".join(
        [k + "='" + v + "'" if isinstance(v, str) \
         else k + "=" + str(v) if isinstance(v, int) \
         else k + "=" + str(v) \
         for k, v in sel_d.items()])
    csql_str = "update " + tab_n + " set " + upg_str + " where " + sel_str
    print(csql_str)
    res = session.execute(csql_str)
    session.shutdown()
    cluster.shutdown()
    return res

cs_update(['192.168.7.110', '192.168.7.111', '192.168.7.61'], 
          "api.ds_services", 
          {"connection": False, "status": "Pending"}, 
          {"ip": "192.168.7.71", "port": 80})

update api.ds_services set connection=False,status='Pending' where ip='192.168.7.71' and port=80


In [10]:
query = "select entity_id from api.data_map group by entity_id, ddn_id"
rows = cs_select(['192.168.7.110', '192.168.7.33'], query)
cs_data = set(row.entity_id for row in rows)
#print(cs_data[:1000])
print(len(cs_data))

1245


In [21]:
CS_HOST_LIST = ["192.168.7.110", "192.168.7.33"]
DS_DELIMITER = "\u0001"

def _get_key_col_str(ip, port, db_str, tab_str):
    tf_loc = DS_DELIMITER.join([ip, str(port), db_str, tab_str])
    query = "select * from scanner.sqltablemeta where tf_loc='" + tf_loc + "'"
    rows = cs_select(CS_HOST_LIST, query) 
    res_set = set(row.primary_key for row in rows)
    primary_key = None
    if res_set:
        primary_key = next(iter(res_set))
    return primary_key


In [22]:
_get_key_col_str("192.168.7.71", 1433, "test", "test")

#### pandas

In [1]:
from cassandra.cluster import Cluster
import pandas as pd

def cs_select(cs_host_list, csql_str):
    cluster = Cluster(cs_host_list)
    session = cluster.connect()
    res = session.execute(csql_str)
    session.shutdown()
    cluster.shutdown()
    return res

def cs_update(cs_host_list, tab_n, upg_d, sel_d):
    cluster = Cluster(cs_host_list)
    session = cluster.connect()
    upg_str = ",".join(
        [k + "='" + v + "'" if isinstance(v, str) \
         else k + "=" + str(v) if isinstance(v, int) \
         else k + "=" + str(v) \
         for k, v in upg_d.items()])
    sel_str = " and ".join(
        [k + "='" + v + "'" if isinstance(v, str) \
         else k + "=" + str(v) if isinstance(v, int) \
         else k + "=" + str(v) \
         for k, v in sel_d.items()])
    csql_str = "update " + tab_n + " set " + upg_str + " where " + sel_str
    res = session.execute(csql_str)
    session.shutdown()
    cluster.shutdown()
    return res

def cs_to_pandas(cs_host_list, csql_str):
    res_cql = list(cs_select(cs_host_list, csql_str))
    res_df = pd.DataFrame(res_cql)
    return res_df

In [8]:
def cs_q_to_edges_df(edge_df_raw):
    edges_df = pd.DataFrame()
    edges_df['from_ddn_id'] = edge_df_raw['ddn_id']
    edges_df['to_ddn_id'] = edge_df_raw['ddn_id']
    edges_df['from_node_id'] = edge_df_raw['id_from']
    edges_df['to_node_id'] = edge_df_raw['id_to']
    attr_col_names = ['total_frequency', 'total_volumn_kb']
    edges_df['edge_attr'] = edge_df_raw.apply(lambda row: dict(zip(attr_col_names, [row[attr_col_name] for attr_col_name in attr_col_names])), axis=1)
    return edges_df

In [10]:
CS_HOST_LIST = ["192.168.7.110", "192.168.7.33"]
this_ddn_id = "54d752f95f5627ffbf66731317db0d20"
query = "select * from api.ddn_edge where ddn_id = '" + this_ddn_id + "' allow filtering"
edge_df_raw = cs_to_pandas(CS_HOST_LIST, query)
edge_df = cs_q_to_edges_df(edge_df_raw)
edge_df.to_dict('records')

[{'edge_attr': {'total_frequency': 100, 'total_volumn_kb': 11270},
  'from_ddn_id': '54d752f95f5627ffbf66731317db0d20',
  'from_node_id': '192.168.8.138',
  'to_ddn_id': '54d752f95f5627ffbf66731317db0d20',
  'to_node_id': '/data/form1.pdf'},
 {'edge_attr': {'total_frequency': 100, 'total_volumn_kb': 11270},
  'from_ddn_id': '54d752f95f5627ffbf66731317db0d20',
  'from_node_id': '192.168.8.138',
  'to_ddn_id': '54d752f95f5627ffbf66731317db0d20',
  'to_node_id': '/data/form2.pdf'},
 {'edge_attr': {'total_frequency': 100, 'total_volumn_kb': 11270},
  'from_ddn_id': '54d752f95f5627ffbf66731317db0d20',
  'from_node_id': '192.168.8.138',
  'to_ddn_id': '54d752f95f5627ffbf66731317db0d20',
  'to_node_id': '/data/form7.pdf'},
 {'edge_attr': {'total_frequency': 100, 'total_volumn_kb': 11270},
  'from_ddn_id': '54d752f95f5627ffbf66731317db0d20',
  'from_node_id': '192.168.8.138',
  'to_ddn_id': '54d752f95f5627ffbf66731317db0d20',
  'to_node_id': '/data/form8.pdf'},
 {'edge_attr': {'total_frequency

In [3]:
import pandas as pd
import numpy as np
from cassandra.cluster import Cluster

In [4]:
def cs_insert(cs_host_list, tab_n, val_dict):
    cluster = Cluster(cs_host_list)
    session = cluster.connect()
    key_str = "(" + ",".join([key for key in val_dict.keys()]) + ")"
    val_str = "(" + ",".join(["%s" for val in val_dict.values()]) + ")"
    cql_str = "insert into " + tab_n + " " + key_str + " values " + val_str
    cql_val_tuple = tuple(val_dict.values())
    session.execute(cql_str, cql_val_tuple)
    session.shutdown()
    cluster.shutdown()

def cs_select(cs_host_list, cql_str, timeout=None):  # timeout is in second
    cluster = Cluster(cs_host_list)
    session = cluster.connect()
    res = session.execute(cql_str, timeout=timeout)
    session.shutdown()
    cluster.shutdown()
    return res

def cs_to_pandas(cs_host_list, cql_str):
    res_cql = list(cs_select(cs_host_list, cql_str))
    res_df = pd.DataFrame(res_cql)
    return res_df

In [5]:
CS_HOST_LIST = ["192.168.7.51"]

In [6]:
#cs_insert(CS_HOST_LIST, "api.test_services", {"storage_type": 0, "ip": "192.168.8.85", "port": 445})

In [11]:
sel_product_category = 1
ts_up = "2019-12-31"
ts_dn = "2018-12-30"
ALERT_COL_NAMES = ['ddn_id', 'product_category', 'story_category', 'story_status', 'story_id', 'timestamp_date']
query = f"select {','.join(ALERT_COL_NAMES)} from api.story_board where product_category={sel_product_category} allow filtering"
story_res = cs_to_pandas(CS_HOST_LIST, query)
if ts_up and ts_dn:
    story_res = story_res[(story_res['timestamp_date'] > ts_dn) & (story_res['timestamp_date'] < ts_up)]
if story_res.empty:
    print("aaa")
story_res = story_res.drop(['timestamp_date'], axis=1)
story_res = story_res.groupby(['ddn_id', 'product_category', 'story_category', 'story_status']).count()
story_res = pd.pivot_table(story_res, values='story_id', index=['ddn_id', 'product_category', 'story_category'], columns=['story_status'], aggfunc=np.sum)
#story_res
story_res = story_res.reset_index().fillna(0)
for x in [1,2,3]:
    if x not in story_res:
        story_res[x] = 0
    story_res[x] = story_res[[x]].astype(int)
story_res = story_res.rename(index=str, columns={1: "a", 2: "b", 3: "c"})
story_res.to_dict('records')

[{'a': 38,
  'b': 0,
  'c': 0,
  'ddn_id': '0925ff44ea703f317da67ffe43bdb226',
  'product_category': 1,
  'story_category': 2},
 {'a': 31,
  'b': 1,
  'c': 0,
  'ddn_id': '55680b95d172d8b3405bb324ab3e2a80',
  'product_category': 1,
  'story_category': 2},
 {'a': 46,
  'b': 0,
  'c': 0,
  'ddn_id': '9c7f9a8642fdc1fcb53a94eb4b413c2d',
  'product_category': 1,
  'story_category': 2},
 {'a': 24,
  'b': 0,
  'c': 0,
  'ddn_id': 'b86fcef380db2ff6b62848cc4309ee65',
  'product_category': 1,
  'story_category': 2},
 {'a': 22,
  'b': 0,
  'c': 0,
  'ddn_id': 'fb422b9eb738989071bbfbd4eacf45ee',
  'product_category': 1,
  'story_category': 2}]

In [14]:
sel_product_category = 1
BOARD_COL_NAMES = ['ddn_id', 'product_category', 'story_category', 'story_id', 'story_status', 'timestamp_date', 'ddn_name', 'node_ids', 'story_status', 'story_string']
ts_up = "2019-12-31"
ts_dn = "2018-12-30"
query = f"select {','.join(BOARD_COL_NAMES)} from api.story_board where product_category={sel_product_category} allow filtering"
story_res = cs_to_pandas(CS_HOST_LIST, query)
if ts_up and ts_dn:
    story_res = story_res[(story_res['timestamp_date'] > ts_dn) & (story_res['timestamp_date'] < ts_up)]
if story_res.empty:
    print("aaa")
story_res = story_res.drop(['timestamp_date'], axis=1)
story_res

,ddn_id,product_category,story_category,story_id,story_status,ddn_name,node_ids,story_status_,story_string
0,55680b95d172d8b3405bb324ab3e2a80,1,2,ff1950859a888fe0de6584f0221a9a2f,1,ENTITY_GROUP-**-9957f9abf70ee5306f12aed73d868a08,(192.168.8.743306customers_infocustomer_new...,1,"User, 89.249.65.30 with ip=89.249.65.30 from d..."
1,55680b95d172d8b3405bb324ab3e2a80,1,2,ff1950859a888fe0de6584f0221a9a2f,1,ENTITY_GROUP-**-9957f9abf70ee5306f12aed73d868a08,(192.168.8.743306customers_infocustomer_new...,1,"User, 89.249.65.30 with ip=89.249.65.30 from d..."
2,55680b95d172d8b3405bb324ab3e2a80,1,2,de4b30c1e5fe5667a5de9a3baafde78d,2,ENTITY_GROUP-**-9957f9abf70ee5306f12aed73d868a08,"(192.168.8.743306customers_infocustomer, 89...",2,"User, 89.249.65.30 with ip=89.249.65.30 from d..."
3,55680b95d172d8b3405bb324ab3e2a80,1,2,de4b30c1e5fe5667a5de9a3baafde78d,1,ENTITY_GROUP-**-9957f9abf70ee5306f12aed73d868a08,"(192.168.8.743306customers_infocustomer, 89...",1,"User, 89.249.65.30 with ip=89.249.65.30 from d..."
4,55680b95d172d8b3405bb324ab3e2a80,1,2,c13c1143a6a8688e151caa2fd7e2b4b2,1,ENTITY_GROUP-**-9957f9abf70ee5306f12aed73d868a08,"(192.168.7.177, 192.168.8.743306customers_in...",1,"User, 192.168.7.177 with ip=192.168.7.177 from..."
5,55680b95d172d8b3405bb324ab3e2a80,1,2,c13c1143a6a8688e151caa2fd7e2b4b2,1,ENTITY_GROUP-**-9957f9abf70ee5306f12aed73d868a08,"(192.168.7.177, 192.168.8.743306customers_in...",1,"User, 192.168.7.177 with ip=192.168.7.177 from..."
6,55680b95d172d8b3405bb324ab3e2a80,1,2,b06bbcf5917c74680384f600aa54f7da,1,ENTITY_GROUP-**-9957f9abf70ee5306f12aed73d868a08,"(185.189.112.89, 192.168.8.743306customers_i...",1,"User, 185.189.112.89 with ip=185.189.112.89 fr..."
7,55680b95d172d8b3405bb324ab3e2a80,1,2,b06bbcf5917c74680384f600aa54f7da,1,ENTITY_GROUP-**-9957f9abf70ee5306f12aed73d868a08,"(185.189.112.89, 192.168.8.743306customers_i...",1,"User, 185.189.112.89 with ip=185.189.112.89 fr..."
8,55680b95d172d8b3405bb324ab3e2a80,1,2,ae4eadea0705d15c45e0e879db9489ef,1,ENTITY_GROUP-**-9957f9abf70ee5306f12aed73d868a08,"(192.168.8.743306customers_infocustomer, 19...",1,"User, 193.176.87.242 with ip=193.176.87.242 fr..."
9,55680b95d172d8b3405bb324ab3e2a80,1,2,ae4eadea0705d15c45e0e879db9489ef,1,ENTITY_GROUP-**-9957f9abf70ee5306f12aed73d868a08,"(192.168.8.743306customers_infocustomer, 19...",1,"User, 193.176.87.242 with ip=193.176.87.242 fr..."
